In [1]:
import os
from src import RASPRoutines
import polars as pl
import numpy as np
from src import IOFunctions

RASP = RASPRoutines.RASP_Routines()
IO = IOFunctions.IO_Functions()

In [ ]:
# MAKE SURE THESE POINT TO THE CORRECT FOLDERS
overall_folder = "20241105_oligomers_in_cells"
raw_data_folder = "sycamore_asap_server/ASAP_Imaging_Data/Main_Survey/Raw_data_ss/"

In [ ]:
brainregion_folders = np.array(
    [
        "putamen",
        "frontal",
        "caudate",
        "cingulate",
        "parietal",
        "parrahippocampal",
        "substantiaNigra",
        "temporal",
    ]
)
cell_stains = np.array(["microglia", "neurons", "astrocytes", "oligodendrocytes"])
patient_status = np.array(["PD", "HC"])
imtype = ".tif"
protein_string = "C0"  # set protein string
cell_string = "C1"  # set cell string
cell_size_threshold = np.array(
    [2000.0, 2000.0, 2000.0, 2000.0]
)  # first one for microglia, second for neurons, third for astrocytes, fourth for oligodendrocytes
percentile = np.array([90.0, 95.0, 99.0])

In [ ]:
for region in brainregion_folders:
    for i, cell_stain in enumerate(cell_stains):
        for j, patient in enumerate(patient_status):
            """the command below will run the analyse_images analysis:

                folder (string): Folder containing images
                imtype (string): Type of images being analysed. Default '.tif'
                thres (float): fraction of bright pixels accepted. Default 0.05.
                large_thres (float): large object intensity threshold. Default 100.
                gisgma (float): gaussian blurring parameter. Default 1.4.
                rwave (float): Ricker wavelent sigma. Default 2.
                protein_string (np.1darray): strings for protein-stained data. Default C1.
                cell_string (np.1darray): strings for cell-containing data. Default C0.
                if_filter (boolean): Filter images for focus. Default True.
                im_start (integer): Images to start from. Default 0.
                cell_analysis (boolean): Parameter where script also analyses cell
                    images and computes colocalisation likelihood ratios. Default True.
                one_savefile (boolean): Parameter that, if true, doesn't save a file. Default True.
                    per image but amalgamates them into one file. Default True.
                disp (boolean): If true, prints when analysed an image stack. Default True.
                analyse_clr (boolean): If true, calculates the clr. If not, just coincidence. Default True.
                error_reduction (boolean): If true, reduces error on the oligomer image using Huang's code

            All have defaults except folder, which is necessary.
            """
            folder = os.path.join(overall_folder, region, cell_stain, patient)
            if j == 0:
                filetest = os.path.join(region, cell_stain, patient+"_analysisparameters", "analysis_params.json")
                if os.path.isfile(filetest):
                    analysis_parameters = IO.load_json(filetest)
                    RASP.flatness = analysis_parameters["flatness"]
                    RASP.integratedGrad = analysis_parameters["integratedGrad"]
                    RASP.areathres = analysis_parameters["areathres"]
                    RASP.d = analysis_parameters["d"]
                    RASP.focus_score_diff = analysis_parameters["focus_score_diff"]
                    RASP.QE = analysis_parameters["QE"]
                    RASP.cell_sigma1 = analysis_parameters["cell_sigma1"]
                    RASP.cell_sigma2 = analysis_parameters["cell_sigma2"]
                    RASP.cell_threshold1 = analysis_parameters["cell_threshold1"]
                    RASP.cell_threshold2 = analysis_parameters["cell_threshold2"]
                else:
                    # first, get the negative controls and calibrate
                    file_list = RASP.file_search(folder, protein_string, imtype)
                    if len(file_list) > 0:
                        example_file = file_list[0]  # first get example file
                        round = "Round" + str(example_file.split("_")[8].split("R")[1])
                        negativecontrol_folder = os.path.join(raw_data_folder, round, "S1")
                        RASP.calibrate_radiality(
                            negativecontrol_folder,
                            imtype=".tif",
                            protein_string=protein_string,
                            gsigma=1.4,
                            rwave=2.0,
                            accepted_ratio=1,
                        )

            # then, run the analyses for the HC and the PD
            filetest_analysis = os.path.join(region, cell_stain, patient+"_analysis", "spot_analysis.csv")
            if not os.path.isfile(filetest_analysis):
                RASP.analyse_images(folder, protein_string=protein_string)

        # then, get the percentile brightness from HC
        analysis_directory = os.path.join(overall_folder, region, cell_stain, "HC" + "_analysis")
        savename = os.path.join(analysis_directory, "spot_analysis.csv")
        if os.path.isfile(savename):
            HC_data = pl.read_csv(savename)
            for p in percentile:
                intensity_threshold = np.percentile(
                    HC_data["sum_intensity_in_photons"].to_numpy(), p
                )
                print("intensity threshold is "+str(np.around(intensity_threshold)))

                for patient in patient_status:
                    analysis_directory = os.path.join(overall_folder, region, cell_stain, patient + "_analysis")
                    analysis_file = os.path.join(
                        os.path.join(analysis_directory,
                        "spot_analysis.csv",
                    )
                    cell_punctum_analysis = (
                        RASP.count_puncta_in_individual_cells_threshold(
                            analysis_file,
                            threshold,
                            cell_size_threshold=cell_size_threshold[i],
                            protein_string=protein_string,
                            cell_string=cell_string,
                        )
                    )